In [1]:
from make_subcube import make_subcube

def create_subcubes_with_make_subcube(path_to_original_cube, output_directory, original_shape, split_shape=(20, 20)):
    """
    Create subcubes from an original FITS cube without splitting along the velocity axis.
    
    Parameters:
    - path_to_original_cube: The file path to the original FITS cube.
    - output_directory: Directory where subcubes will be saved.
    - original_shape: A tuple of the shape of the original cube (z, y, x).
    - split_shape: A tuple indicating how many times the cube should be split along the RA and Dec axes.
    """
    z, y, x = original_shape
    sy, sx = split_shape
    y_step, x_step = y // sy, x // sx

    # No loop for the velocity axis; it remains intact
    for j in range(sy):
        for k in range(sx):
            y_start, y_end = j * y_step, (j + 1) * y_step if j < sy - 1 else y
            x_start, x_end = k * x_step, (k + 1) * x_step if k < sx - 1 else x

            # Build slice_params for each subcube
            slice_params = [slice(None), slice(y_start, y_end), slice(x_start, x_end)]
            
            # Use make_subcube to extract and save each subcube
            subcube_hdu = make_subcube(slice_params=slice_params, path_to_file=path_to_original_cube, 
                                       save=True, overwrite=True, 
                                       path_to_output_file=f"{output_directory}/subcube_{j}_{k}.fits",
                                       get_hdu=True)

            print(f"Subcube saved: sub_cube_{j}_{k}.fits")

def merge_subcubes_into_original(path_to_subcubes, original_shape, output_file, split_shape=(4, 4)):
    """
    Read processed subcubes and merge them into a single FITS cube without splitting along the velocity axis.
    
    Parameters:
    - input_dir: Directory containing the processed subcubes.
    - original_shape: The shape of the original FITS cube (z, y, x).
    - output_file: Path to save the merged FITS cube.
    - split_shape: The split size used when the original cube was divided, excluding the velocity axis.
    """
    z, y, x = original_shape
    sy, sx = split_shape
    merged_cube = np.zeros(original_shape, dtype=np.float32)

    for j in range(sy):
        for k in range(sx):
            # subcube_2_1_g+_fit_fin_sf-p1_decomp
            subcube_path = f"{path_to_subcubes}/subcube_{j}_{k}_g+_fit_fin_sf-p1_decomp.fits"
            with fits.open(subcube_path) as hdul:
                subcube_data = hdul[0].data
                # Calculate the starting indices for y and x axes
                y_start, x_start = j * (y // sy), k * (x // sx)
                y_end, x_end = y_start + subcube_data.shape[1], x_start + subcube_data.shape[2]
                # No need to calculate for z axis since it's not split
                # Place the subcube data into the correct position of the merged cube
                merged_cube[:, y_start:y_end, x_start:x_end] = subcube_data

    # Save the merged cube to a file
    return merged_cube
    # fits.writeto(output_file, merged_cube, overwrite=True)
    # print(f"Merged cube saved to: {output_file}")



In [2]:
create_subcubes_with_make_subcube('~/GALCAR_cutout_repro.fits', './subcubes/',[270, 1731, 1263], split_shape=(2, 2)  )


making subcube with the slice parameters [slice(None, None, None), slice(0, 865, None), slice(0, 631, None)]...
Subcube saved: sub_cube_0_0.fits

making subcube with the slice parameters [slice(None, None, None), slice(0, 865, None), slice(631, 1263, None)]...
Subcube saved: sub_cube_0_1.fits

making subcube with the slice parameters [slice(None, None, None), slice(865, 1731, None), slice(0, 631, None)]...
Subcube saved: sub_cube_1_0.fits

making subcube with the slice parameters [slice(None, None, None), slice(865, 1731, None), slice(631, 1263, None)]...
Subcube saved: sub_cube_1_1.fits


In [7]:
import numpy as np
from astropy.io import fits
merged_cube=merge_subcubes_into_original('FITS',[270, 1731, 1263] , 'output/')

In [8]:
from astropy.io import fits

# 打开FITS文件进行更新
with fits.open('output/A_cutout_1.21_tofang_maskrfi_mask_complete.fits', mode='update') as hdul:
    # hdul是一个HDU列表对象，每个HDU代表FITS文件中的一个数据部分

    # 访问主HDU（Header/Data Unit），通常是列表的第一个元素
    hdul[0].data=merged_cube
